<a href="https://colab.research.google.com/github/srJboca/segmentacion/blob/main/EN/1.%20Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Exploring Customer and Gas Billing Data

## Introduction

Welcome to this tutorial on data exploration. We will work with a dataset that simulates information from a gas distribution company. The objective is to clean, combine, and explore this data to better understand customers, their consumption, and payment behaviors. This process is fundamental before performing more advanced tasks such as customer segmentation, predictive consumption modeling, or delinquency analysis.

The data is divided into four files:
1.  `clientes.parquet`: Information about customers (contracts, demographic data).
2.  `facturas.parquet`: Details of issued invoices (consumption, dates).
3.  `precios_gas.parquet`: Gas prices per m³ according to stratum, year, and month.
4.  `recaudo.parquet`: Information about payments made for the invoices.

## Phase 1: Environment Setup and Data Loading

### 1.1 Importing Libraries

First, we will import the necessary libraries.
* `pandas` for data manipulation and analysis.
* `matplotlib.pyplot` and `seaborn` for data visualization.
* `warnings` to manage any warnings that may arise.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Settings for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2 Downloading the Data Files

We will download the Parquet files from the GitHub repository.

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/clientes.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/facturas.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/precios_gas.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/recaudo.parquet

### 1.3 Loading Data into Pandas DataFrames

Now, we will load each Parquet file into a Pandas DataFrame.

In [ ]:
df_clients = pd.read_parquet('clientes.parquet')
df_invoices = pd.read_parquet('facturas.parquet')
df_collections = pd.read_parquet('recaudo.parquet')
df_gas_prices = pd.read_parquet('precios_gas.parquet')

## Phase 2: Initial Data Inspection

We will perform a basic inspection of each DataFrame to understand its structure, data types, and check for null values or initial descriptive statistics.

### 2.1 Clients DataFrame (`df_clients`)

In [ ]:
print("--- df_clients Information ---")
df_clients.info()
print("\n--- First 5 rows of df_clients ---")
print(df_clients.head())
print("\n--- Null values in df_clients ---")
print(df_clients.isnull().sum())
print("\n--- Descriptive statistics of df_clients ---")
print(df_clients.describe(include='all'))
print("\n--- Count of unique values per column in df_clients ---")
for col in df_clients.columns:
    print(f"Column '{col}': {df_clients[col].nunique()} unique values")

**Observations on `df_clients`:**
* Contains personal and contractual information of the clients.
* Columns like `Ciudad` and `Estrato socioeconomico` are categorical and could be important for segmentation.
* The date columns are object types; we will need to convert them to `datetime`.
* `Numero de contrato` appears to be the unique identifier.

### 2.2 Invoices DataFrame (`df_invoices`)

In [ ]:
print("--- df_invoices Information ---")
df_invoices.info()
print("\n--- First 5 rows of df_invoices ---")
print(df_invoices.head())
print("\n--- Null values in df_invoices ---")
print(df_invoices.isnull().sum())
print("\n--- Descriptive statistics of df_invoices ---")
print(df_invoices.describe(include='all'))
print("\n--- Count of unique values per column in df_invoices ---")
for col in df_invoices.columns:
    print(f"Column '{col}': {df_invoices[col].nunique()} unique values")

**Observations on `df_invoices`:**
* Contains details of each invoice, including consumption and relevant dates.
* `Numero de factura` is the identifier for the invoice, and `Numero de contrato` links it to the client.
* `Consumo (m3)` is a key numerical variable.
* The date columns are also object types.

### 2.3 Collections DataFrame (`df_collections`)

In [ ]:
print("--- df_collections Information ---")
df_collections.info()
print("\n--- First 5 rows of df_collections ---")
print(df_collections.head())
print("\n--- Null values in df_collections ---")
print(df_collections.isnull().sum())
print("\n--- Descriptive statistics of df_collections ---")
print(df_collections.describe(include='all'))
print("\n--- Count of unique values per column in df_collections ---")
for col in df_collections.columns:
    print(f"Column '{col}': {df_collections[col].nunique()} unique values")

**Observations on `df_collections`:**
* Records the date on which an invoice was paid.
* Joins with `df_invoices` using `Numero de factura`.
* `Fecha de Pago Real` will need conversion to `datetime`.

### 2.4 Gas Prices DataFrame (`df_gas_prices`)

In [ ]:
print("--- df_gas_prices Information ---")
df_gas_prices.info()
print("\n--- First 5 rows of df_gas_prices ---")
print(df_gas_prices.head())
print("\n--- Null values in df_gas_prices ---")
print(df_gas_prices.isnull().sum())
print("\n--- Descriptive statistics of df_gas_prices ---")
print(df_gas_prices.describe(include='all'))
print("\n--- Count of unique values per column in df_gas_prices ---")
for col in df_gas_prices.columns:
    print(f"Column '{col}': {df_gas_prices[col].nunique()} unique values")

**Observations on `df_gas_prices`:**
* Defines the price per m³ of gas according to `Año`, `Mes`, and `Estrato`.
* `Precio m3 (COP)` is the numerical price value.
* `Estrato` is categorical.

## Phase 3: Merging the Data

To get a consolidated view, we will combine these DataFrames.

### 3.1 Joining Invoices with Customer Information

We join `df_invoices` with `df_clients` using `Numero de contrato` as the key.

In [ ]:
df_invoice_client = pd.merge(df_invoices, df_clients, on='Numero de contrato', how='left')
print("--- df_invoice_client Information (Invoices + Clients) ---")
df_invoice_client.info()
print("\n--- First 5 rows of df_invoice_client ---")
print(df_invoice_client.head())
print(f"\nShape of df_invoices: {df_invoices.shape}")
print(f"Shape of df_invoice_client: {df_invoice_client.shape}")

**Verification:** The number of rows should be equal to that of `df_invoices` if each invoice has a corresponding customer (using `how='left'`). If it increases, it could indicate duplicates in `df_clients` by `Numero de contrato`, which should not happen.

### 3.2 Adding Prices to Invoices

Now we join `df_invoice_client` with `df_gas_prices`. The join is done using `Año`, `Mes`, and the customer's socioeconomic stratum.

In [ ]:
df_invoice_client_price = pd.merge(df_invoice_client,
                                     df_gas_prices,
                                     left_on=['Año', 'Mes', 'Estrato socioeconomico'],
                                     right_on=['Año', 'Mes', 'Estrato'],
                                     how='left')

print("--- df_invoice_client_price Information (Invoices + Clients + Prices) ---")
df_invoice_client_price.info()
print("\n--- First 5 rows of df_invoice_client_price ---")
print(df_invoice_client_price.head())
print(f"\nShape of df_invoice_client: {df_invoice_client.shape}")
print(f"Shape of df_invoice_client_price: {df_invoice_client_price.shape}")

# Check for rows where the price could not be assigned (NaN in 'Precio m3 (COP)')
print("\n--- Rows without an assigned price ---")
print(df_invoice_client_price[df_invoice_client_price['Precio m3 (COP)'].isnull()].head())

**Verification and Potential Issues:**
* Again, the number of rows should be consistent. If it increases drastically, it might indicate that the join keys `['Año', 'Mes', 'Estrato socioeconomico']` are not unique in `df_gas_prices` for a given combination, or that there are multiple invoices for the same client in the same month with the same stratum being mapped (which is expected).
* The `Estrato` column from `df_gas_prices` is redundant after the merge and can be dropped.
* It's crucial to check if there are any invoices to which a price could not be assigned. This could happen if a combination of `Año, Mes, Estrato socioeconomico` from the invoices does not exist in the price table.

### 3.3 Incorporating Payment Information (Collections)

Finally, we join the payment information from `df_collections`.

In [ ]:
df_complete = pd.merge(df_invoice_client_price,
                       df_collections,
                       on='Numero de factura',
                       how='left') # We use a left join to keep all invoices, even if they don't have a recorded payment

print("--- df_complete Information (All data joined) ---")
df_complete.info()
print("\n--- First 5 rows of df_complete ---")
print(df_complete.head())
print(f"\nShape of df_invoice_client_price: {df_invoice_client_price.shape}")
print(f"Shape of df_complete: {df_complete.shape}")

# Check for invoices without an actual payment date
print("\n--- Invoices without actual payment date (NaN in 'Fecha de Pago Real') ---")
print(df_complete[df_complete['Fecha de Pago Real'].isnull()][['Numero de factura', 'Fecha de Emision', 'Fecha de Pago Oportuno']].head())

**Verification:**
* The number of rows should remain the same or increase if an invoice has multiple payment records (uncommon, unless they are non-consolidated partial payments). A `left join` ensures all invoices are kept.
* Invoices without a corresponding `Fecha de Pago Real` will have `NaN` in that column. This is expected for unpaid invoices or those whose payment has not yet been recorded.

### 3.4 Column Selection and Handling Duplicates

The final analysis DataFrame will use a subset of columns. We will select these columns and then handle any potential duplicates that may have been generated during the merges, especially if the keys were not perfectly unique or if the same information was joined multiple times.

In [ ]:
selected_columns = [
    'Numero de factura', 'Numero de contrato', 'Fecha de Emision', 'Consumo (m3)',
    'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Suspension Estimada',
    'Ciudad', 'Estrato socioeconomico',
    'Precio m3 (COP)', 'Fecha de Pago Real'
]

if 'Estrato' in df_complete.columns and 'Estrato socioeconomico' in df_complete.columns:
    df_complete = df_complete.drop(columns=['Estrato'])

# Create the analysis DataFrame with selected columns
df_analysis = df_complete[selected_columns].copy()

print("--- First rows of df_analysis (before handling duplicates) ---")
print(df_analysis.head())
print(f"\nShape of df_analysis before drop_duplicates: {df_analysis.shape}")

num_duplicates_before = df_analysis.duplicated().sum()
print(f"Number of exact duplicate rows before: {num_duplicates_before}")

df_analysis = df_analysis.drop_duplicates()
print(f"\nShape of df_analysis after drop_duplicates: {df_analysis.shape}")
num_duplicates_after = df_analysis.duplicated().sum()
print(f"Number of exact duplicate rows after: {num_duplicates_after}")

print("\n--- First rows of df_analysis (after handling duplicates) ---")
print(df_analysis.head())
df_analysis.info()

**Note on Duplicates:**
The `drop_duplicates()` step is crucial to ensure the integrity of the analysis. It's important to investigate *why* duplicates were generated (e.g., non-unique merge keys? source data with duplicates?). For now, we have removed them.

## Phase 4: Feature Engineering

We will create new columns from existing ones to enrich our analysis.

### 4.1 Date Conversion

We convert the date columns to `datetime` format.

In [ ]:
date_cols = ['Fecha de Emision', 'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Pago Real', 'Fecha de Suspension Estimada']
for col in date_cols:
    df_analysis[col] = pd.to_datetime(df_analysis[col], errors='coerce')

print("--- Data types after converting dates ---")
df_analysis.info()

### 4.2 Calculating the Invoice Amount (Price per Consumption)

In [ ]:
df_analysis['Price_per_Consumption'] = df_analysis['Precio m3 (COP)'] * df_analysis['Consumo (m3)']
print("\n--- df_analysis with 'Price_per_Consumption' ---")
print(df_analysis[['Consumo (m3)', 'Precio m3 (COP)', 'Price_per_Consumption']].head())

### 4.3 Calculating Time Differences

We will calculate the number of days between different key events.

In [ ]:
df_analysis['Days_Issue_DueDate'] = (df_analysis['Fecha de Pago Oportuno'] - df_analysis['Fecha de Emision']).dt.days
df_analysis['Days_Reading_Issue'] = (df_analysis['Fecha de Emision'] - df_analysis['Fecha de Lectura']).dt.days
df_analysis['Days_DueDate_ActualPayment'] = (df_analysis['Fecha de Pago Real'] - df_analysis['Fecha de Pago Oportuno']).dt.days

print("\n--- df_analysis with new days features ---")
print(df_analysis[['Fecha de Emision', 'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Pago Real',
                   'Days_Issue_DueDate', 'Days_Reading_Issue', 'Days_DueDate_ActualPayment']].head())

### 4.4 Identifying Late Payments

We create a binary column `Late_Payment`: 1 if the payment was made after the due date, 0 if it was paid on time or before. It is considered 0 if there is no actual payment date (NaT).

In [ ]:
df_analysis['Late_Payment'] = 0
df_analysis.loc[df_analysis['Days_DueDate_ActualPayment'] > 0, 'Late_Payment'] = 1
df_analysis.loc[df_analysis['Days_DueDate_ActualPayment'].isnull(), 'Late_Payment'] = 0 # Assume not late if not paid yet

print("\n--- df_analysis with the 'Late_Payment' column ---")
print(df_analysis[['Fecha de Pago Oportuno', 'Fecha de Pago Real', 'Days_DueDate_ActualPayment', 'Late_Payment']].head(10))
print("\nValue counts in 'Late_Payment':")
print(df_analysis['Late_Payment'].value_counts(dropna=False))

## Phase 5: Detailed Exploration of the Consolidated DataFrame (`df_analysis`)

Now that we have a clean and enriched DataFrame, we can explore it in more depth.

### 5.1 General Summary and Null Values

In [ ]:
print("--- General information of df_analysis ---")
df_analysis.info()

print("\n--- Null values in df_analysis ---")
print(df_analysis.isnull().sum())

print("\n--- Descriptive statistics of df_analysis ---")
print(df_analysis.describe(include='all'))

**Observations on null values:**
* `Fecha de Suspension Estimada`: May have nulls if not all invoices have this date.
* `Precio m3 (COP)` and `Price_per_Consumption`: Nulls here would indicate problems in the merge with `df_gas_prices` or missing data in the price table.
* `Fecha de Pago Real` and `Days_DueDate_ActualPayment`: Nulls are expected for unpaid invoices.

It's important to decide how to handle these nulls. For some variables, they could be imputed; for others (like `Fecha de Pago Real`), their absence is informative.

### 5.2 Distribution of Key Numerical Variables

In [ ]:
numerical_cols_to_plot = ['Consumo (m3)', 'Precio m3 (COP)', 'Price_per_Consumption',
                           'Days_Issue_DueDate', 'Days_Reading_Issue', 'Days_DueDate_ActualPayment']

for col in numerical_cols_to_plot:
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    sns.histplot(df_analysis[col].dropna(), kde=True, bins=30) # dropna() to avoid errors with NaT/NaN in histplot
    plt.title(f'Distribution of {col}')

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df_analysis[col].dropna())
    plt.title(f'Boxplot of {col}')

    plt.tight_layout()
    plt.show()
    print(f"Statistics for {col}:\n{df_analysis[col].describe()}\n")

**Interpretation of distributions:**
* **Consumo (m3):** Observe the shape of the distribution. Is it symmetrical, skewed? Are there outliers (very high or low consumption)?
* **Precio m3 (COP):** Does it vary much? This will depend on the socioeconomic strata and temporal evolution.
* **Price_per_Consumption:** Similar to consumption, but scaled by price.
* **Days Variables:**
    * `Days_Issue_DueDate`: The time allowed for payment. Is it constant?
    * `Days_Reading_Issue`: Time between meter reading and invoice issuance. Is it consistent?
    * `Days_DueDate_ActualPayment`: Positive values indicate late payment. Negative values, early payment. Zero, payment on the due date. The boxplot can show the magnitude of the delay or advance.

### 5.3 Distribution of Key Categorical Variables

In [ ]:
categorical_cols_to_plot = ['Ciudad', 'Estrato socioeconomico', 'Late_Payment']

for col in categorical_cols_to_plot:
    plt.figure(figsize=(8, 5))
    sns.countplot(data=df_analysis, x=col, order=df_analysis[col].value_counts(dropna=False).index)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)
    plt.show()
    print(f"Counts for {col}:\n{df_analysis[col].value_counts(dropna=False)}\n")

**Interpretation of categorical distributions:**
* **Ciudad:** How are the invoices/customers distributed by city?
* **Estrato socioeconomico:** Which strata are the most common?
* **Late_Payment:** What proportion of invoices are paid late?

### 5.4 Correlation Analysis (Numerical Variables)

In [ ]:
numerical_df = df_analysis.select_dtypes(include=['number'])

plt.figure(figsize=(12, 8))
correlation_matrix = numerical_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

print("\n--- Pairs with highest (absolute) correlation ---")
corr_pairs = correlation_matrix.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)
# Filter for relevance (> 0.5) and remove self-correlation and duplicates
unique_corr_pairs = sorted_pairs[(abs(sorted_pairs) < 1) & (abs(sorted_pairs) > 0.5)].drop_duplicates()
print(unique_corr_pairs.head(10))

**Interpretation of the Correlation Matrix:**
* Look for correlation coefficients close to 1 (strong positive correlation) or -1 (strong negative correlation).
* For example, a high correlation is expected between `Consumo (m3)` and `Price_per_Consumption`.
* Unexpected correlations can reveal interesting insights.

### 5.5 Relationships between Variables

Let's explore some specific relationships.

#### 5.5.1 Average Consumption by Stratum and City

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=df_analysis, x='Estrato socioeconomico', y='Consumo (m3)', hue='Ciudad', estimator=pd.Series.mean, errorbar=None, order=sorted(df_analysis['Estrato socioeconomico'].dropna().unique()))
plt.title('Average Consumption (m3) by Socioeconomic Stratum and City')
plt.ylabel('Average Consumption (m3)')
plt.xlabel('Socioeconomic Stratum')
plt.xticks(rotation=45)
plt.legend(title='City')
plt.show()

plt.figure(figsize=(14, 7))
sns.boxplot(data=df_analysis, x='Estrato socioeconomico', y='Consumo (m3)', hue='Ciudad', order=sorted(df_analysis['Estrato socioeconomico'].dropna().unique()))
plt.title('Distribution of Consumption (m3) by Socioeconomic Stratum and City')
plt.ylabel('Consumption (m3)')
plt.xlabel('Socioeconomic Stratum')
plt.xticks(rotation=45)
plt.legend(title='City')
plt.show()

#### 5.5.2 Late Payment Rate by Stratum and City

In [ ]:
late_payment_by_stratum_city = df_analysis.groupby(['Estrato socioeconomico', 'Ciudad'])['Late_Payment'].mean().reset_index()
late_payment_by_stratum_city = late_payment_by_stratum_city.rename(columns={'Late_Payment': 'Late_Payment_Rate'})

plt.figure(figsize=(12, 6))
sns.barplot(data=late_payment_by_stratum_city, x='Estrato socioeconomico', y='Late_Payment_Rate', hue='Ciudad', order=sorted(late_payment_by_stratum_city['Estrato socioeconomico'].dropna().unique()))
plt.title('Late Payment Rate by Socioeconomic Stratum and City')
plt.ylabel('Late Payment Rate (Proportion)')
plt.xlabel('Socioeconomic Stratum')
plt.xticks(rotation=45)
plt.legend(title='City')
plt.show()

### 5.6 Temporal Analysis (Example: Consumption over time)

In [ ]:
df_temporal = df_analysis.set_index('Fecha de Emision').sort_index()

monthly_consumption = df_temporal['Consumo (m3)'].resample('ME').mean()

plt.figure(figsize=(14, 7))
monthly_consumption.plot(marker='o', linestyle='-')
plt.title('Average Monthly Gas Consumption (m3) Over Time')
plt.xlabel('Issue Date (Month)')
plt.ylabel('Average Consumption (m3)')
plt.grid(True)
plt.show()

monthly_invoices = df_temporal.resample('ME').size()
plt.figure(figsize=(14, 7))
monthly_invoices.plot(kind='bar')
plt.title('Number of Invoices Issued per Month')
plt.xlabel('Issue Date (Month)')
plt.ylabel('Number of Invoices')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

**Interpretation of Temporal Analysis:**
* Are there trends in consumption (increase, decrease)?
* Is seasonality observed (peaks in certain months)?
* The number of invoices can indicate growth in the customer base or fluctuations in billing.

## Phase 6: Exploration Conclusions and Next Steps

In this exploratory phase, we have:
1.  Loaded and performed an initial inspection of four data sources.
2.  Combined the data into a unified DataFrame (`df_analysis`).
3.  Identified and handled duplicate rows that arose during merges.
4.  Performed feature engineering, creating new variables like `Price_per_Consumption`, time differences, and a `Late_Payment` indicator.
5.  Explored the distributions of key numerical and categorical variables.
6.  Analyzed correlations and relationships between variables (e.g., consumption by stratum, late payment by stratum).
7.  Conducted a brief temporal analysis of consumption.